In [155]:
import pandas as pd
import pickle


### Explicacion general:
Hemos tenido que adaptar los datos para que puedan ser exportados a PowerBI. De inicio no nos acepta todo el df por el peso del archivo. PWBI solo acepta hasta 1M de observaciones por csv. Asi que separaremos el df original en distintos df independientes.

Hemos tratado de agrupar los datos de forma mensual para poder comprimir los datos

La particion la haremos siguiendo dos lineas, UNIDADES y VALOR DE VENTA.

Finalmente tendremos 8 archivos en total:
- UNITS
    - PWBI_summary_units
    - PWBI_ny_units
    - PWBI_boston_units
    - PWBI_phila_units

- VALUE
    - PWBI_df_monthly_summary
    - PWBI_df_monthly_ny
    - PWBI_df_monthly_boston
    - PWBI_df_monthly_phila

In [156]:
df=pd.read_csv('datos_arreglados.csv').drop('Unnamed: 0',axis=1)
pd.set_option('display.float_format', '{:.2f}'.format)
df

,date,item,category,department,stores,region,week,year,is_holidays,units_sold,sell_price,total_sales
0,2013-07-08,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,South_End(BOS_1),Boston,28,2013,0,0,12.74,0.00
1,2013-07-15,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,South_End(BOS_1),Boston,28,2013,1,0,12.74,0.00
2,2013-07-22,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,South_End(BOS_1),Boston,29,2013,0,4,10.99,43.94
3,2013-07-29,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,South_End(BOS_1),Boston,30,2013,0,6,10.99,65.91
4,2013-08-05,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,South_End(BOS_1),Boston,31,2013,0,0,10.99,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...
6610508,2016-03-28,SUPERMARKET_3_827,SUPERMARKET,SUPERMARKET_3,Queen_Village(PHI_3),Philadelphia,12,2016,1,22,1.20,26.40
6610509,2016-04-04,SUPERMARKET_3_827,SUPERMARKET,SUPERMARKET_3,Queen_Village(PHI_3),Philadelphia,13,2016,0,17,1.20,20.40
6610510,2016-04-11,SUPERMARKET_3_827,SUPERMARKET,SUPERMARKET_3,Queen_Village(PHI_3),Philadelphia,14,2016,0,6,1.20,7.20
6610511,2016-04-18,SUPERMARKET_3_827,SUPERMARKET,SUPERMARKET_3,Queen_Village(PHI_3),Philadelphia,15,2016,0,0,1.20,0.00


In [157]:
df.groupby('year')['total_sales'].sum()

year
2011   29308613.52
2012   39865952.47
2013   44834525.31
2014   46529779.55
2015   51150358.95
2016   17919419.13
Name: total_sales, dtype: float64

In [158]:
# Creamos dos nuevas columnas que nos seran utiles para el analisis de PowerBI
df['month'] = pd.to_datetime(df['date']).dt.strftime('%m')
df['month_year'] =  pd.to_datetime(df['date']).dt.strftime('%B %Y')
# cambiamos estructura tipo de dato para que powerbi lo trague (con floats tiene problemas)
df['total_sales']=df['total_sales'].round(2).astype(int)
df['sell_price']=df['sell_price'].round(2)

df

,date,item,category,department,stores,region,week,year,is_holidays,units_sold,sell_price,total_sales,month,month_year
0,2013-07-08,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,South_End(BOS_1),Boston,28,2013,0,0,12.74,0,07,July 2013
1,2013-07-15,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,South_End(BOS_1),Boston,28,2013,1,0,12.74,0,07,July 2013
2,2013-07-22,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,South_End(BOS_1),Boston,29,2013,0,4,10.99,43,07,July 2013
3,2013-07-29,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,South_End(BOS_1),Boston,30,2013,0,6,10.99,65,07,July 2013
4,2013-08-05,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,South_End(BOS_1),Boston,31,2013,0,0,10.99,0,08,August 2013
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6610508,2016-03-28,SUPERMARKET_3_827,SUPERMARKET,SUPERMARKET_3,Queen_Village(PHI_3),Philadelphia,12,2016,1,22,1.20,26,03,March 2016
6610509,2016-04-04,SUPERMARKET_3_827,SUPERMARKET,SUPERMARKET_3,Queen_Village(PHI_3),Philadelphia,13,2016,0,17,1.20,20,04,April 2016
6610510,2016-04-11,SUPERMARKET_3_827,SUPERMARKET,SUPERMARKET_3,Queen_Village(PHI_3),Philadelphia,14,2016,0,6,1.20,7,04,April 2016
6610511,2016-04-18,SUPERMARKET_3_827,SUPERMARKET,SUPERMARKET_3,Queen_Village(PHI_3),Philadelphia,15,2016,0,0,1.20,0,04,April 2016


In [159]:
df.groupby('year')['total_sales'].sum()
# Verificamos que no hemos perdido muchos datos por pasar de float a integer

year
2011    29018273
2012    39480903
2013    44361466
2014    45993017
2015    50563263
2016    17720173
Name: total_sales, dtype: int32

In [160]:
# Creamos un df que agrupara los datos por mes con la intencion de reducir el peso de los datos y que power BI lo trague (acepta maximo 1 millon de filas)
df_monthly= df.groupby(['item','category','department','stores','region','year','month','month_year','is_holidays','sell_price'])['total_sales'].sum().reset_index()

df_monthly



,item,category,department,stores,region,year,month,month_year,is_holidays,sell_price,total_sales
0,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,Back_Bay(BOS_3),Boston,2013,07,July 2013,0,11.46,0
1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,Back_Bay(BOS_3),Boston,2013,08,August 2013,0,11.46,77
2,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,Back_Bay(BOS_3),Boston,2013,09,September 2013,0,11.46,89
3,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,Back_Bay(BOS_3),Boston,2013,09,September 2013,1,11.46,57
4,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,Back_Bay(BOS_3),Boston,2013,10,October 2013,0,11.46,89
...,...,...,...,...,...,...,...,...,...,...,...
2788330,SUPERMARKET_3_827,SUPERMARKET,SUPERMARKET_3,Yorktown(PHI_2),Philadelphia,2016,02,February 2016,0,1.20,83
2788331,SUPERMARKET_3_827,SUPERMARKET,SUPERMARKET_3,Yorktown(PHI_2),Philadelphia,2016,02,February 2016,1,1.20,62
2788332,SUPERMARKET_3_827,SUPERMARKET,SUPERMARKET_3,Yorktown(PHI_2),Philadelphia,2016,03,March 2016,0,1.20,76
2788333,SUPERMARKET_3_827,SUPERMARKET,SUPERMARKET_3,Yorktown(PHI_2),Philadelphia,2016,03,March 2016,1,1.20,16


In [161]:
# Creamos otro df para las ventass por volumen(unidades)
df_monthly_units= df.groupby(['item','category','department','stores','region','year','month','month_year','is_holidays'])['units_sold'].sum().reset_index()

df_monthly_units



,item,category,department,stores,region,year,month,month_year,is_holidays,units_sold
0,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,Back_Bay(BOS_3),Boston,2013,07,July 2013,0,0
1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,Back_Bay(BOS_3),Boston,2013,08,August 2013,0,7
2,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,Back_Bay(BOS_3),Boston,2013,09,September 2013,0,8
3,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,Back_Bay(BOS_3),Boston,2013,09,September 2013,1,5
4,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,Back_Bay(BOS_3),Boston,2013,10,October 2013,0,8
...,...,...,...,...,...,...,...,...,...,...
2731897,SUPERMARKET_3_827,SUPERMARKET,SUPERMARKET_3,Yorktown(PHI_2),Philadelphia,2016,02,February 2016,0,70
2731898,SUPERMARKET_3_827,SUPERMARKET,SUPERMARKET_3,Yorktown(PHI_2),Philadelphia,2016,02,February 2016,1,52
2731899,SUPERMARKET_3_827,SUPERMARKET,SUPERMARKET_3,Yorktown(PHI_2),Philadelphia,2016,03,March 2016,0,65
2731900,SUPERMARKET_3_827,SUPERMARKET,SUPERMARKET_3,Yorktown(PHI_2),Philadelphia,2016,03,March 2016,1,14


In [162]:
# seguimos dividiedo los datos por ciudad y unidades

df_monthly_boston_units= df_monthly_units[df_monthly_units['region']=='Boston']
df_monthly_phila_units= df_monthly_units[df_monthly_units['region']=='Philadelphia']
df_monthly_ny_units= df_monthly_units[df_monthly_units['region']=='New York']

# printeamos los shapes para ver si todo esta bien y tragaria el PwBI (menos de 1M de filas)
print(df_monthly_boston_units.shape)
print(df_monthly_phila_units.shape)
print(df_monthly_ny_units.shape)
df_monthly_ny_units

# El unico que excede el tamaño maximo es el df_monthly_ny_units

(835663, 10)
(815287, 10)
(1080952, 10)


,item,category,department,stores,region,year,month,month_year,is_holidays,units_sold
60,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,Brooklyn(NYC_4),New York,2013,07,July 2013,0,4
61,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,Brooklyn(NYC_4),New York,2013,07,July 2013,1,0
62,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,Brooklyn(NYC_4),New York,2013,08,August 2013,0,8
63,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,Brooklyn(NYC_4),New York,2013,09,September 2013,0,12
64,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,Brooklyn(NYC_4),New York,2013,09,September 2013,1,5
...,...,...,...,...,...,...,...,...,...,...
2731864,SUPERMARKET_3_827,SUPERMARKET,SUPERMARKET_3,Tribeca(NYC_3),New York,2016,02,February 2016,0,286
2731865,SUPERMARKET_3_827,SUPERMARKET,SUPERMARKET_3,Tribeca(NYC_3),New York,2016,02,February 2016,1,219
2731866,SUPERMARKET_3_827,SUPERMARKET,SUPERMARKET_3,Tribeca(NYC_3),New York,2016,03,March 2016,0,283
2731867,SUPERMARKET_3_827,SUPERMARKET,SUPERMARKET_3,Tribeca(NYC_3),New York,2016,03,March 2016,1,34


In [163]:
# Eliminaremos los valores aporten menos al total del analisis(Valores menores a 2 y que no sean dias festivos)
print('El las unidades totales a eliminar del df es: ', df_monthly_ny_units[(df_monthly_ny_units['units_sold']<2 )&(df_monthly_ny_units['is_holidays']==0)]['units_sold'].sum(),'units')

df_monthly_ny_units= df_monthly_ny_units[~((df_monthly_ny_units['units_sold'] < 2) & (df_monthly_ny_units['is_holidays'] == 0))]
df_monthly_ny_units

# y ahora si podemos introducir los datos en power BI

El las unidades totales a eliminar del df es:  19958 units


,item,category,department,stores,region,year,month,month_year,is_holidays,units_sold
60,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,Brooklyn(NYC_4),New York,2013,07,July 2013,0,4
61,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,Brooklyn(NYC_4),New York,2013,07,July 2013,1,0
62,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,Brooklyn(NYC_4),New York,2013,08,August 2013,0,8
63,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,Brooklyn(NYC_4),New York,2013,09,September 2013,0,12
64,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,Brooklyn(NYC_4),New York,2013,09,September 2013,1,5
...,...,...,...,...,...,...,...,...,...,...
2731864,SUPERMARKET_3_827,SUPERMARKET,SUPERMARKET_3,Tribeca(NYC_3),New York,2016,02,February 2016,0,286
2731865,SUPERMARKET_3_827,SUPERMARKET,SUPERMARKET_3,Tribeca(NYC_3),New York,2016,02,February 2016,1,219
2731866,SUPERMARKET_3_827,SUPERMARKET,SUPERMARKET_3,Tribeca(NYC_3),New York,2016,03,March 2016,0,283
2731867,SUPERMARKET_3_827,SUPERMARKET,SUPERMARKET_3,Tribeca(NYC_3),New York,2016,03,March 2016,1,34


In [164]:
# para poder ver el una imagen global , vamos a sectorizar por categoria departamento ciudad y tienda en UNITS.

df_monthly_summary_units= df_monthly_units.groupby(['category','department','stores','region','year','month','is_holidays','month_year'])['units_sold'].sum().reset_index()
df_monthly_summary_units

,category,department,stores,region,year,month,is_holidays,month_year,units_sold
0,ACCESORIES,ACCESORIES_1,Back_Bay(BOS_3),Boston,2011,01,1,January 2011,145
1,ACCESORIES,ACCESORIES_1,Back_Bay(BOS_3),Boston,2011,01,1,January 2012,1163
2,ACCESORIES,ACCESORIES_1,Back_Bay(BOS_3),Boston,2011,02,0,February 2011,2122
3,ACCESORIES,ACCESORIES_1,Back_Bay(BOS_3),Boston,2011,02,1,February 2011,2169
4,ACCESORIES,ACCESORIES_1,Back_Bay(BOS_3),Boston,2011,03,0,March 2011,4360
...,...,...,...,...,...,...,...,...,...
7930,SUPERMARKET,SUPERMARKET_3,Yorktown(PHI_2),Philadelphia,2016,02,0,February 2016,44606
7931,SUPERMARKET,SUPERMARKET_3,Yorktown(PHI_2),Philadelphia,2016,02,1,February 2016,32655
7932,SUPERMARKET,SUPERMARKET_3,Yorktown(PHI_2),Philadelphia,2016,03,0,March 2016,51439
7933,SUPERMARKET,SUPERMARKET_3,Yorktown(PHI_2),Philadelphia,2016,03,1,March 2016,13631


In [165]:
# guardamos los df en csv:

df_monthly_summary_units.to_csv('PWBI_summary_units.csv')
df_monthly_boston_units.to_csv('PWBI_boston_units.csv')
df_monthly_phila_units.to_csv('PWBI_phila_units.csv')
df_monthly_ny_units.to_csv('PWBI_ny_units.csv')

In [166]:
# AHORA HACEMOS LO MISMO PERO CON LAS VENTAS
# seguimos dividiedo los datos por ciudad y ventas totales

df_monthly_boston= df_monthly[df_monthly['region']=='Boston']
df_monthly_phila= df_monthly[df_monthly['region']=='Philadelphia']
df_monthly_ny= df_monthly[df_monthly['region']=='New York']

# printeamos los shapes para ver si todo esta bien y tragaria el PwBI (menos de 1M de filas)
print(df_monthly_boston.shape)
print(df_monthly_phila.shape)
print(df_monthly_ny.shape)
df_monthly_ny

# el unico que lo supera y por poco es el de NY

(855309, 11)
(830530, 11)
(1102496, 11)


,item,category,department,stores,region,year,month,month_year,is_holidays,sell_price,total_sales
61,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,Brooklyn(NYC_4),New York,2013,07,July 2013,0,12.74,50
62,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,Brooklyn(NYC_4),New York,2013,07,July 2013,1,12.74,0
63,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,Brooklyn(NYC_4),New York,2013,08,August 2013,0,10.99,85
64,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,Brooklyn(NYC_4),New York,2013,09,September 2013,0,10.99,128
65,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,Brooklyn(NYC_4),New York,2013,09,September 2013,1,10.99,54
...,...,...,...,...,...,...,...,...,...,...,...
2788297,SUPERMARKET_3_827,SUPERMARKET,SUPERMARKET_3,Tribeca(NYC_3),New York,2016,02,February 2016,0,1.20,343
2788298,SUPERMARKET_3_827,SUPERMARKET,SUPERMARKET_3,Tribeca(NYC_3),New York,2016,02,February 2016,1,1.20,262
2788299,SUPERMARKET_3_827,SUPERMARKET,SUPERMARKET_3,Tribeca(NYC_3),New York,2016,03,March 2016,0,1.20,339
2788300,SUPERMARKET_3_827,SUPERMARKET,SUPERMARKET_3,Tribeca(NYC_3),New York,2016,03,March 2016,1,1.20,40


In [167]:

df_monthly_ny['total_sales'].describe()

count   1102496.00
mean         92.67
std         213.75
min           0.00
25%           9.00
50%          38.00
75%          99.00
max       26919.00
Name: total_sales, dtype: float64

In [168]:
# Eliminaremos los valores aporten menos al total del analisis(Valores menores a 6 y que no sean dias festivos)
print('El valor total a eliminar del df es: ', df_monthly_ny[(df_monthly_ny['total_sales']<=6 )&(df_monthly_ny['is_holidays']==0)]['total_sales'].sum().round(2),'$')

df_monthly_ny= df_monthly_ny[~((df_monthly_ny['total_sales'] <=6) & (df_monthly_ny['is_holidays'] == 0))]
df_monthly_ny

# y ahora si podemos introducir los datos en power BI

El valor total a eliminar del df es: 

 100198 $


,item,category,department,stores,region,year,month,month_year,is_holidays,sell_price,total_sales
61,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,Brooklyn(NYC_4),New York,2013,07,July 2013,0,12.74,50
62,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,Brooklyn(NYC_4),New York,2013,07,July 2013,1,12.74,0
63,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,Brooklyn(NYC_4),New York,2013,08,August 2013,0,10.99,85
64,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,Brooklyn(NYC_4),New York,2013,09,September 2013,0,10.99,128
65,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,Brooklyn(NYC_4),New York,2013,09,September 2013,1,10.99,54
...,...,...,...,...,...,...,...,...,...,...,...
2788297,SUPERMARKET_3_827,SUPERMARKET,SUPERMARKET_3,Tribeca(NYC_3),New York,2016,02,February 2016,0,1.20,343
2788298,SUPERMARKET_3_827,SUPERMARKET,SUPERMARKET_3,Tribeca(NYC_3),New York,2016,02,February 2016,1,1.20,262
2788299,SUPERMARKET_3_827,SUPERMARKET,SUPERMARKET_3,Tribeca(NYC_3),New York,2016,03,March 2016,0,1.20,339
2788300,SUPERMARKET_3_827,SUPERMARKET,SUPERMARKET_3,Tribeca(NYC_3),New York,2016,03,March 2016,1,1.20,40


In [169]:
# para poder ver el una imagen global , vamos a sectorizar por categoria departamento ciudad y tienda.

df_monthly_summary= df_monthly.groupby(['category','department','stores','region','year','month','is_holidays','month_year'])['total_sales'].sum().reset_index()
df_monthly_summary

,category,department,stores,region,year,month,is_holidays,month_year,total_sales
0,ACCESORIES,ACCESORIES_1,Back_Bay(BOS_3),Boston,2011,01,1,January 2011,616
1,ACCESORIES,ACCESORIES_1,Back_Bay(BOS_3),Boston,2011,01,1,January 2012,5145
2,ACCESORIES,ACCESORIES_1,Back_Bay(BOS_3),Boston,2011,02,0,February 2011,10257
3,ACCESORIES,ACCESORIES_1,Back_Bay(BOS_3),Boston,2011,02,1,February 2011,10394
4,ACCESORIES,ACCESORIES_1,Back_Bay(BOS_3),Boston,2011,03,0,March 2011,20473
...,...,...,...,...,...,...,...,...,...
7930,SUPERMARKET,SUPERMARKET_3,Yorktown(PHI_2),Philadelphia,2016,02,0,February 2016,116729
7931,SUPERMARKET,SUPERMARKET_3,Yorktown(PHI_2),Philadelphia,2016,02,1,February 2016,85299
7932,SUPERMARKET,SUPERMARKET_3,Yorktown(PHI_2),Philadelphia,2016,03,0,March 2016,134785
7933,SUPERMARKET,SUPERMARKET_3,Yorktown(PHI_2),Philadelphia,2016,03,1,March 2016,34764


In [170]:
# guardamos los datos en un csv para poder importarlos en PowerBI

df_monthly_summary.to_csv('PWBI_df_monthly_summary.csv')
df_monthly_boston.to_csv('PWBI_df_monthly_boston.csv')
df_monthly_phila.to_csv('PWBI_df_monthly_phila.csv')
df_monthly_ny.to_csv('PWBI_df_monthly_ny.csv')

In [171]:
# vamos a ver que los totales de cada df coinciden con los totales de los datos de PWBI
print('Total ventas Boston: ',df_monthly_boston['total_sales'].sum().round(2))
print('Total ventas Philadelphia: ',df_monthly_phila['total_sales'].sum().round(2))
print('Total ventas New York: ',df_monthly_ny['total_sales'].sum().round(2))
print('Total ventas global: ',df_monthly['total_sales'].sum().round(2))

Total ventas Boston:  65421935
Total ventas Philadelphia:  59543581
Total ventas New York:  102071381
Total ventas global:  227137095


In [172]:
df_monthly_summary.groupby('year')['total_sales'].sum()

year
2011    29018273
2012    39480903
2013    44361466
2014    45993017
2015    50563263
2016    17720173
Name: total_sales, dtype: int32

float normal.
Total ventas Boston:  66171408.85
Total ventas Philadelphia:  60263420.79
Total ventas New York:  103171040.12
Total ventas global:  229608722.1

float a integer
Total ventas Boston:  65420608.0
Total ventas Philadelphia:  59542054.0
Total ventas New York:  102167634.0
Total ventas global:  227132313.0

float a integer tras aproximacion
Total ventas Boston:  66180915.0
Total ventas Philadelphia:  60273308.0
Total ventas New York:  103185031.0
Total ventas global:  229640938.0

